In [ ]:
'''
Goal:Plan out markers for MERScope

Author:Carsten Knutsen
Date:240117
conda_env:uterus_sc
Notes: Create figures for initial rnascope 
'''

In [ ]:
import scanpy as sc

# Only needed for processing
import numpy as np
import pandas as pd
import os 
import scanpy.external as sce
pd.set_option('display.max_rows', 500)
import liana as li

output = '/home/carsten/alvira_bioinformatics/uterus/data/pilot/240325_merscope_planning'
os.makedirs(output, exist_ok=True)
sc.set_figure_params(dpi=300, format="png")
sc.settings.figdir = output


In [ ]:
def deg_priority(deg_fn,n_top_genes,level_col,fc_col,level_thresh=50,fc_thresh=1):
    ## Make sure fn is sorted by stat
    final_dict={}
    df_dict = pd.read_excel(deg_fn,sheet_name=None,header=0,index_col=0)
    for sheet in df_dict.keys():
        df = df_dict[sheet].copy()
        if level_thresh == None:
            pass
        else:
            df = df.loc[df[level_col] > level_thresh]
        df = df.loc[abs(df[fc_col])>fc_thresh].head(n_top_genes)
        final_dict[sheet] =df.sort_values(fc_col).index.tolist()
    return final_dict
        
        

In [ ]:
def produce_rnascope_graphs(adata,title,obs,groups,genes,output):
    os.makedirs(output, exist_ok=True)
    sc.settings.figdir = output
    adata_cts = adata[adata.obs[obs].isin(groups)]
    sc.pl.umap(adata,color=genes,save=f'_{title}_all')
    sc.pl.dotplot(adata,genes,groupby=[obs],save=f'_{title}_all.png')
    sc.pl.dotplot(adata_cts,genes,groupby=[obs],save=f'_{title}_specifc_celltypes.png')

In [ ]:
def subcluster_celltype(adata, celltype,output_fol):
    output_ct = f'{output_fol}/subcluster/{celltype}'
    os.makedirs(output_ct, exist_ok=True)
    sc.set_figure_params(dpi=300, format="png")
    sc.settings.figdir = output_ct
    lin_adata = adata[adata.obs['Cell Subtype']==celltype]
    try:
        sc.pp.highly_variable_genes(lin_adata,
                                    batch_key="Patient"
                                    )
    except:
        sc.pp.highly_variable_genes(lin_adata
                            )
    sc.pp.pca(lin_adata, use_highly_variable=True)
    try:
        sce.pp.harmony_integrate(lin_adata, key="Patient", max_iter_harmony=50)
        sc.pp.neighbors(lin_adata, use_rep='X_pca_harmony')

    except:
        sc.pp.neighbors(lin_adata)

    sc.tl.leiden(
        lin_adata,
        key_added=f"leiden_{celltype}",
    )
    sc.tl.umap(lin_adata,min_dist=0.1)
    sc.tl.rank_genes_groups(lin_adata, f"leiden_{celltype}",pts=True, method="wilcoxon")
    print(lin_adata.obs[f"leiden_{celltype}"].cat.categories)
    sc.pl.rank_genes_groups_dotplot(
        lin_adata,
        groupby=f"leiden_{celltype}",
        n_genes=int(100 / len(lin_adata.obs[f"leiden_{celltype}"].unique())),
        show=False,
        save=f"{celltype}_leiden_markers.png",
    )

    for color in [f'leiden_{celltype}','Patient','GroupContract','Cell Subtype']:
        sc.pl.umap(lin_adata, color = color, show=False,save=color)
    with pd.ExcelWriter(
        f"{output_ct}/{celltype}_leiden_markers.xlsx", engine="xlsxwriter") as writer:
        for ld in sorted(lin_adata.obs[f"leiden_{celltype}"].unique()):
            df = sc.get.rank_genes_groups_df(
                lin_adata, key="rank_genes_groups", group=ld
            )
            df.to_excel(writer, sheet_name=f"{ld} v rest"[:31])
        lin_adata.write(f'{output_ct}/{celltype}_adata.gz.h5ad', compression='gzip')

In [ ]:
def comparsion_dictionary_flip(comp_dict):
    gene_dict={}
    for comp in comp_dict.keys():
        ct_dict=comp_dict[comp]
        for ct in ct_dict.keys():
            gene_ls = ct_dict[ct]
            for gene in gene_ls:
                if gene in gene_dict.keys():
                    gene_dict[gene].append(f'{comp}:{ct}')
                else:
                    gene_dict[gene] = [f'{comp}:{ct}']
    return gene_dict


In [ ]:
adata = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped.gz.h5ad')
adata.raw = adata
adata.uns['log1p']['base'] =10

In [ ]:
# ## Subcluster each cell type
# for celltype in adata.obs['Cell Subtype'].unique():
#     print(celltype)
#     subcluster_celltype(adata,celltype,output)

In [ ]:
# Get broad lineage markers
sc.settings.figdir = output
sc.tl.rank_genes_groups(adata, f"Lineage",pts=True, method="wilcoxon")
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby=f"Lineage",
    n_genes=int(20 / len(adata.obs[f"Lineage"].unique())),
    show=False,
    save=f"20_Lineage_markers.png",
)


lin_marker_ls = []
lin_marker_dt={}
for ct in adata.obs[f"Lineage"].unique():
    df = sc.get.rank_genes_groups_df(
        adata, key="rank_genes_groups", group=ct
    )
    df['pct_exp_dif']= df['pct_nz_group'] - df['pct_nz_reference']
#     df = df.sort_values('pct_exp_dif',ascending=False)
    lin_marker_dt[ct]=df['names'].head(10).values
    lin_marker_ls+=list(df['names'].head(10).values)
sc.pl.dotplot(adata,lin_marker_dt,groupby='Lineage',dendrogram=True,save='top10_markers_per_Lineage.png')
lin_marker_ls = []
lin_marker_dt={}
for ct in adata.obs[f"Lineage"].unique():
    df = sc.get.rank_genes_groups_df(
        adata, key="rank_genes_groups", group=ct
    )
    df['pct_exp_dif']= df['pct_nz_group'] - df['pct_nz_reference']
    df = df.sort_values('pct_exp_dif',ascending=False)
    lin_marker_dt[ct]=df['names'].head(10).values
    lin_marker_ls+=list(df['names'].head(10).values)
sc.pl.dotplot(adata,lin_marker_dt,groupby='Lineage',dendrogram=True,save='top10_markers_per_Lineage_pct.png')

In [ ]:
ct_marker_dt={}
ct_marker_ls=[]
for lineage in adata.obs['Lineage'].cat.categories:
    deg_fn = f'/home/carsten/alvira_bioinformatics/uterus/data/figures/tissue_embedding/{lineage}/{lineage}_Cell Subtype_markers.xlsx'
    marker_dt = pd.read_excel(deg_fn,sheet_name=None,index_col=0, header=0)
    for sheet in marker_dt.keys():
        df = marker_dt[sheet]
        ct = sheet.split('v rest')[0].strip(' ')
        df['pct_exp_dif']= df['pct_nz_group'] - df['pct_nz_reference']
        df = df.sort_values('pct_exp_dif',ascending=False)
        ct_marker_dt[ct]=list(df.head(10).index)
        ct_marker_ls+=list(df.head(10).index)
sc.pl.dotplot(adata,ct_marker_dt,groupby='Cell Subtype',dendrogram=True,save='top10_markers_per_Cell Subtype_inlin.png')

In [ ]:
# ## Get broad cell type markers

# sc.settings.figdir = output
# sc.tl.rank_genes_groups(adata, f"Cell Subtype",pts=True, method="wilcoxon")
# sc.pl.rank_genes_groups_dotplot(
#     adata,
#     groupby=f"Cell Subtype",
#     n_genes=int(200 / len(adata.obs[f"Cell Subtype"].unique())),
#     show=False,
#     save=f"200_Cell Subtype_markers.png",
# )


# ct_marker_ls = []
# ct_marker_dt={}
# for ct in adata.obs[f"Cell Subtype"].unique():
#     df = sc.get.rank_genes_groups_df(
#         adata, key="rank_genes_groups", group=ct
#     )
#     df['pct_exp_dif']= df['pct_nz_group'] - df['pct_nz_reference']
#     df = df.sort_values('pct_exp_dif',ascending=False)
#     ct_marker_dt[ct]=df['names'].head(10).values
#     ct_marker_ls+=list(df['names'].head(10).values)
# sc.pl.dotplot(adata,ct_marker_dt,groupby='Cell Subtype',dendrogram=True,save='top10_markers_per_Cell Subtype.png')

In [ ]:
## Top DEGs for each cell type
deg_fol_path = f'/home/carsten/alvira_bioinformatics/uterus/data/figures/deg/pseudobulk'
comp_dict={}
for fn_s in [x for x in os.listdir(deg_fol_path) if x.endswith('.xlsx')]:
    fn = f'{deg_fol_path}/{fn_s}'
    label = fn_s.split('.')[0]
    output_fol = f'{output}/deg/{label}'
    os.makedirs(output_fol, exist_ok=True)
    sc.settings.figdir = output_fol
    comp_n = label.split('_')
    comps = ['-'.join(comp_n[:2]), '-'.join(comp_n[3:5])]
    deg_dict = deg_priority(fn,20,'baseMean','log2FoldChange')
    comp_dict[label] = deg_dict
    for ct in deg_dict.keys():
        genes = deg_dict[ct]
        if len(genes)==0:
            continue
        ct_adata = adata[(adata.obs['Cell Subtype']==ct)
                       &(adata.obs['GroupContract'].isin(comps))
                       ]
        sc.pp.scale(ct_adata)
        sc.pl.dotplot(ct_adata, 
                      genes,
                      groupby='GroupContract',
                      title=f'{comps[0]} v {comps[1]} in {ct}',
                      show=False,
                      save = f'{ct}_top20_lfcgt1.png'
                     )
        
        

In [ ]:
comp_dict

In [ ]:
ct_distinguish = [
    'ADGRL3',
    'ARHGAP15',
    'JAG1',
    'DPP6',
    'LINGO2',
    'RBFOX3',
    'KCNB1',
    'ACTA2',
    'NOTCH3',
    'GJA5',
    'NOSTRIN',
    'PROX1',
    'PDGFRA',
    'RORB',
    'CCDC80',
    'RGS5',
    'RGS6'
                 ]

In [ ]:
immune_markers_vizgen = [
    'FCER2',
    'BLK',
    'CD19',
    'CD22',
    'CD79A',
    'CD8A',
    'CD8B',
    'CD160',
    'CD244',
    'CD3D',
    'CD3E',
    'MARCO',
    'CD207',
    'CMKLR1',
    'IL23A',
    'CD163',
    'FCER1A',
    'FCGR2A',
    'FCGR3A',
    'GPX3',
    'FGFBP2',
    'GZMH',
    'KIR2DL4',
    'KLRF1',
    'IFNGR1',
    'FGF1',
    'IL1R2',
    'FGFR1',
    'IL2RB',
    'FGFR2',
    'IL3RA',
    'CXCL16',
    'IFNGR2',
    'PROK2',
    'EGF',
    'IL17A',
    'BTLA',
    'CD274',
    'CTLA4',
    'HAVCR2',
    'LAG3',
    'PDCD1',
    'PDCD1LG2',
    'TIGIT'
]
##https://info.vizgen.com/io-panel
lin_adata=adata[adata.obs['Lineage']=='Immune',immune_markers_vizgen]
lin_adata.obs['Cell Subtype'] = lin_adata.obs['Cell Subtype'].astype('str')
sc.pl.dotplot(lin_adata,
              immune_markers_vizgen,
             groupby='Cell Subtype',
             )

In [ ]:
proliferative_markers = ['DIAPH3',
'CENPK',
'BRIP1',
'BRCA2',
'TOP2A',
                         'MKI67',
                         'CDKN1A',
                         'CDKN2A',
                         
]

In [ ]:
cfrna_sci_ls = ['CLCN3', 'DAPP1', 'PPBP', 'MAP3K7CL', 'MOB1B', 'RAB27B','RGS18']

In [ ]:
cfrna_nat_ls = ['CAMK2G','DERA','FAM46A','KIAA1109','LRRC58','MYLIP','NDIFV3','NMRK1','PI4KA','PRTFDC1','PYGO2','RNF149','TFIP11','TRIM21','USB1','YWHAQP5','ENST00000391229','ENST00000364542']

In [ ]:
manual_remove = [
'AC012593.1',
 'AC024230.1',
 'AC060234.3',
 'AC063944.1',
 'AC079352.1',
 'AC079793.1',
 'AC079804.3',
 'AC092979.1',
 'AC104078.2',
 'AC110767.1',
    'AC012404.1',
 'AC092683.1',
 'AC139720.1',
     'AL078604.4',
 'AL136456.1',
 'AL163541.1',
 'AL357507.1',
    'AP001977.1',
 'AP002518.2',
     'HBB',
    'LINC00621',
 'LINC01088',
 'LINC01320',
 'LINC01605',
 'LINC01764',
 'LINC02147',
 'LINC02532',
    'ADAMTS9-AS2',
     'AF233439.1',
    'AL359636.2',
 'AL365214.2',
    'LINC01091',
 'LINC01913',
 'LINC02197',
     'MALAT1',
     'PAX8-AS1',




]

In [ ]:
jess_genes = pd.read_excel(f'{output}/Jess genes/Jess genes for MERscope.xlsx',index_col=None,header=None)[0].values.tolist()
jess_genes = sorted(set(jess_genes))

In [ ]:
daiana_genes = pd.read_excel(f'{output}/Daiana Genes/List Daiana.xlsx',index_col=None,header=None)
daiana_genes = daiana_genes[1].dropna(how='any')
daiana_genes = daiana_genes.str.upper().str.replace(' ','').values.tolist()
daiana_genes = sorted(set(daiana_genes))

In [ ]:
comp_dict['ct_marker'] = ct_marker_dt
comp_dict['lin_marker'] = lin_marker_dt

In [ ]:

gene_dict = comparsion_dictionary_flip(comp_dict)
gene_dict
gene_lists_key = {'subcluster_marker':ct_distinguish,
                'vizgen_immune':immune_markers_vizgen,
                 'select_proliferative':proliferative_markers,
                 'Jess_genes':jess_genes,
                 'Daiana_genes':daiana_genes,
                  'cfRNA_science':cfrna_sci_ls,
                  'cfRNA_nature':cfrna_nat_ls,

                  
                
                }
gene_lists_key.keys()

gene_dict_cp = gene_dict.copy()
gene_dict_cp
for key in gene_lists_key.keys():
    ls=sorted(set(gene_lists_key[key]))
    for gene in ls:
        if gene in gene_dict_cp.keys():
            gene_dict_cp[gene].append(key)
        else:
            gene_dict_cp[gene] = [key]
gene_dict_cp

final_dict = {}
for key in gene_dict_cp.keys():
    ls = [x for x in gene_dict_cp[key]if not x.startswith('TNL')]
    if len(ls) !=0:
        final_dict[key] = ls
final_dict


rename_dict = {
    'CAMKII':'CAMK2A',
              'CD66':'CEACAM1',
              'IP3':'ITPR3',
              'LINCO1088':'LINC01088',
              'MAGL1':'MGLL',
               'MLCK':'MYLK',
               'NRF2-AS1':'NR2F2-AS1',
               'ORAI':'ORAI1',
               'PDGFRB18':'PDGFRB',
               'SAMD':'SAMD9',
               'SERCA':'ATP2A2',
               'STA1':'SAT1',
               'STIM':'STIM1',
               'CALM':'CALM3',
               'CALN':'PPP3CA',
               'CAMKI':'CAMK1',
               'CK7':'KRT7',
               'CD56':'NCAM1',
    'FAM46A':'TENT5A',
               'KCA':'CSN3',
               'MEK1':'MAP2K1',
               'PKC':'PRRT2',
               'SLP1':'SYTL1',
    'GPCR':'ENSG00000220349',
    'HLADQB1':'HLA-DQB1',
    'HLAG':'HLA-G',
    'NDIFV3':'NDUFS3',    
               
              }
for k,v in rename_dict.items():
    if v in final_dict.keys():
        final_dict[v] = sorted(set(final_dict[v]+final_dict[k]))
        final_dict.pop(k)
    else:
        final_dict[v] = final_dict.pop(k)


final_sr = pd.Series(final_dict, name='Reason')
final_sr.index.name = 'Gene'
final_sr= final_sr.sort_index()
final_sr.to_csv(f'{output}/gene_list.csv')

rm_ls = [
    
    'A2ML1-AS1',
'AF233439.1',
'AP002518.2',
'AC079352.1',
'AL022068.1',
    'RTK',
    'ACTA2',
    'TAGLN',
    'DES',
    'MYL9',
    'ACTG2',
    'MGP',
    'B2M',
    'SPARCL1',
    'TPM2',
'NEAT1',
    'TPSB2',
    'TPSAB1',
    'MT1A',
    'AC110767.1'

]
trim_ls = final_sr.loc[[x for x in final_sr.index if x not in rm_ls]]
trim_ls.to_csv(f'{output}/gene_list_trim.csv')

In [ ]:
overlap = [x for x in trim_ls.index if x in adata.var_names]
print([x for x in trim_ls.index if x not in overlap])
lin_adata = adata[:,overlap]
sc.pp.pca(lin_adata, use_highly_variable=False)

# sce.pp.harmony_integrate(lin_adata, key="Patient", max_iter_harmony=50)
# sc.pp.neighbors(lin_adata, use_rep='X_pca_harmony')

sc.pp.neighbors(lin_adata, use_rep='X_pca')

sc.tl.leiden(
    lin_adata,
)
sc.tl.umap(lin_adata,min_dist=0.1)
sc.pl.umap(lin_adata,color = 'Cell Subtype')
sc.pl.umap(lin_adata,color = 'Lineage')

In [ ]:
for ct in adata.obs['Cell Subtype'].cat.categories:
    sc.pl.umap(lin_adata, color = 'Cell Subtype', groups=[ct],title=ct)

In [ ]:
sc.pl.umap(lin_adata,color = 'log10_n_genes_by_umis')


In [ ]:
sc.pl.umap(lin_adata,color='Patient')

In [ ]:
output_umap = f'{output}/gene_ls_figs'
os.makedirs(output_umap, exist_ok=True)
sc.settings.figdir = output_umap

for gene in lin_adata.var_names:
    sc.pl.umap(adata,color =gene,save=f'_{gene}.png',show=False)

In [ ]:
fpkm_sheet = pd.read_csv('/home/carsten/Downloads/gene_fpkm.csv')
fpkm_sheet.index = fpkm_sheet['gene_name']
fpkm_sheet = fpkm_sheet[['pt22', 'pt24', 'pt36', 'pt37', 'pt41', 'pt33', 'pt4',
       'pt11', 'pt13', 'pt21', 'pt25', 'pt38']]

fpkm_sheet = fpkm_sheet.T.mean()
fpkm_trim = fpkm_sheet.loc[[x for x in trim_ls.index if x in fpkm_sheet.index]]

In [ ]:
[x for x in trim_ls.index if x not in fpkm_trim.index]

In [ ]:
fpkm_trim.to_csv(f'{output}/fpkm_ls')

In [ ]:



# total_ls = ct_marker_ls+lin_marker_ls+ct_distinguish+immune_markers_vizgen + proliferative_markers
# trim_total = [x for x in total_ls if x not in manual_remove]
# trim_total = sorted(set(trim_total))
# print(len(trim_total))




# pd.Series(trim_total).to_csv(f'{output}/gene_ls.csv')
# pd.Series(gene_dict).to_csv(f'{output}/gene_dict_stats.csv')
# trim_total